
# A Complete Persona Pattern Example

In our previous tutorial, we built an **Invoice Processing Agent** that could extract and store structured invoice data.  
Now, we will enhance this system by introducing **expert consultations** to handle two critical tasks:

1. **Categorizing Expenditures**: The agent will consult an expert to classify each invoice into one of 20 predefined spending categories based on a one-sentence description.  
2. **Checking Purchasing Rules**: The agent will validate whether the invoice follows company purchasing policies.  

By using expert tools, we enable our agent to dynamically invoke specialized knowledge without hardcoding every rule, keeping the architecture clean and adaptable.  


## Step 1: Creating the Invoice Categorization Expert

In [ ]:

@register_tool(tags=["invoice_processing", "categorization"])
def categorize_expenditure(action_context: ActionContext, description: str) -> str:
    """
    Categorize an invoice expenditure based on a short description.
    """
    categories = [
        "Office Supplies", "IT Equipment", "Software Licenses", "Consulting Services", 
        "Travel Expenses", "Marketing", "Training & Development", "Facilities Maintenance",
        "Utilities", "Legal Services", "Insurance", "Medical Services", "Payroll",
        "Research & Development", "Manufacturing Supplies", "Construction", "Logistics",
        "Customer Support", "Security Services", "Miscellaneous"
    ]
    
    return prompt_expert(
        action_context=action_context,
        description_of_expert="A senior financial analyst with deep expertise in corporate spending categorization.",
        prompt=f"Given the following description: '{description}', classify the expense into one of these categories:\n{categories}"
    )


## Step 2: Creating the Purchasing Rules Expert

In [ ]:

@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context: ActionContext, invoice_data: dict) -> dict:
    """
    Validate an invoice against company purchasing policies.
    """
    rules_path = "config/purchasing_rules.txt"
    
    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        purchasing_rules = "No rules available. Assume all invoices are compliant."

    return prompt_expert(
        action_context=action_context,
        description_of_expert="A corporate procurement compliance officer with extensive knowledge of purchasing policies.",
        prompt=f"""
        Given this invoice data: {invoice_data}, check whether it complies with company purchasing rules.
        The latest purchasing rules are as follows:
        
        {purchasing_rules}
        
        Identify any violations or missing requirements. Respond with:
        - "compliant": true or false
        - "issues": A brief explanation of any problems found
        """
    )



### Example: Updating Purchasing Rules Without Code Changes

Suppose our rules file contains:

```
1. All purchases over $5,000 require pre-approval.
2. IT equipment purchases must be from approved vendors.
3. Travel expenses must include a justification.
4. Consulting fees over $10,000 require an SOW (Statement of Work).
```

**Invoice Data Example:**


In [ ]:

invoice_data = {
    "invoice_number": "7890",
    "date": "2025-03-10",
    "vendor": "Tech Solutions Inc.",
    "total_amount": 6000,
    "line_items": [
        {"description": "High-end workstation", "quantity": 1, "total": 6000}
    ]
}


## Generating Structured Responses Rather than Free Text

In [ ]:

@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context: ActionContext, invoice_data: dict) -> dict:
    """
    Validate an invoice against company purchasing policies, returning a structured response.
    """
    rules_path = "config/purchasing_rules.txt"

    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        purchasing_rules = "No rules available. Assume all invoices are compliant."

    validation_schema = {
        "type": "object",
        "properties": {
            "compliant": {"type": "boolean"},
            "issues": {"type": "string"}
        }
    }

    return prompt_llm_for_json(
        action_context=action_context,
        schema=validation_schema,
        prompt=f"""
        Given this invoice data: {invoice_data}, check whether it complies with company purchasing rules.
        The latest purchasing rules are as follows:
        
        {purchasing_rules}
        
        Respond with a JSON object containing:
        - `compliant`: true if the invoice follows all policies, false otherwise.
        - `issues`: A brief explanation of any violations or missing requirements.
        """
    )


## Step 3: Updating the Invoice Processing Agent

In [ ]:

def create_invoice_agent():
    action_registry = PythonActionRegistry()

    goals = [
        Goal(
            name="Persona",
            description="You are an Invoice Processing Agent, specialized in handling invoices efficiently."
        ),
        Goal(
            name="Process Invoices",
            description="""
            Your goal is to process invoices accurately. For each invoice:
            1. Extract key details such as vendor, amount, and line items.
            2. Generate a one-sentence summary of the expenditure.
            3. Categorize the expenditure using an expert.
            4. Validate the invoice against purchasing policies.
            5. Store the processed invoice with categorization and validation status.
            6. Return a summary of the invoice processing results.
            """
        )
    ]

    environment = PythonEnvironment()

    return Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=environment
    )


## Step 4: Testing the New Capabilities

In [ ]:

invoice_text = """
    Invoice #4567
    Date: 2025-02-01
    Vendor: Tech Solutions Inc.
    Items: 
      - Laptop - $1,200
      - External Monitor - $300
    Total: $1,500
"""

agent = create_invoice_agent()
response = agent.run(f"Process this invoice:\n\n{invoice_text}")
print(response)
